In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data_train = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Dataset/train.csv")
df_v = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Dataset/validation.csv")
#df_test = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Dataset/test.csv")

df_predict = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Github/Web-Economics-master/click_pred.csv")

df_bidprice = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Github/Web-Economics-master/out.csv")

df_v[df_v.bidid == 'd939cd3025fba6ee162860e5cb4fad21']

df_predict.rename(columns={' click': 'click'}, inplace=True)

df_predict.click_v.sum()

#211 clicks predicted / 16 accurate
df_predict[(df_predict.click ==1)&(df_predict.click_v == 1)].shape[0]/df_predict[df_predict.click_v == 1].shape[0]

#### Bidding model

In [4]:
#Take a quarter of the trainning set as a sample 
data_train = data_train.sample(frac = 0.6,random_state = 20).reset_index(drop=True)

In [5]:
def CTR_calc(data):
    Click_t_rate= "Click Through Rate= "+ str(round((np.count_nonzero(data.click.values)/data.shape[0])*100,4)) + ' %'
    return (Click_t_rate)

In [6]:
data_train.shape

(1618643, 26)

In [7]:
CTR_calc(data_train)

'Click Through Rate= 0.0773 %'

In [8]:
#Dropping un-interesting columns
data_train= data_train.drop(['bidid','userid','logtype','url','urlid','keypage','bidprice','payprice'],1)

In [9]:
data_train.columns

Index(['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag'],
      dtype='object')

In [10]:
#Second drop- to improve performance 
#data_train = data_train.drop(['IP','domain','usertag'],1)

In [11]:
data_train.head()

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0,5,10,windows_ie,119.249.43.*,3,9,3,5F1RQS9rg5scFsf,News_F_Width1,1000,90,0,0,80,832b91d59d0cb5731431653204a76c0e,1458,"13042,10076,10031,10057,10075,10006,13403,1006..."
1,0,1,16,windows_ie,110.241.12.*,3,4,3,3c47edf60cc139641b6395e123a6e155,ALLINONE_F_Width1,1000,90,Na,Na,70,10734,2821,"14273,10102,10006,10110,10031,13403,10063"
2,0,0,23,windows_ie,61.146.117.*,216,230,3,7ed515fe566938ee6cfbb6ebb7ea4995,Sports_F_Upright,300,250,Na,Na,10,10722,2821,"10059,10684,10083,10024,10006,16706,10111,1005..."
3,0,1,18,windows_ie,112.15.2.*,94,104,3,3043163ba84753b4b51dd3290caeae67,tips_F_Width2,1000,90,Na,Na,50,10734,2821,null
4,0,1,3,windows_ie,223.241.165.*,106,114,3,3SCYZrn0Qo18XMB4JKTI,discuz_18316225_005,728,90,0,0,20,b90c12ed2bd7950c6027bf9c6937c48a,3386,null


### Preprocessing of data

In [12]:
#Removing strings in useragent
data_train.update(data_train['useragent'].str.replace('_ie', ''))
data_train.update(data_train['useragent'].str.replace('_firefox', ''))
data_train.update(data_train['useragent'].str.replace('_safari', ''))
data_train.update(data_train['useragent'].str.replace('_chrome', ''))
data_train.update(data_train['useragent'].str.replace('_other', ''))
data_train.update(data_train['useragent'].str.replace('_opera', ''))
data_train.update(data_train['useragent'].str.replace('_maxthon', ''))
data_train.update(data_train['useragent'].str.replace('_theworld', ''))
data_train.update(data_train['useragent'].str.replace('_sogou', ''))

#### Encoding the data

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le=LabelEncoder()

In [15]:
le_ad= LabelEncoder()

In [16]:
le_ad.fit(['1458', '2821', '3386', '2261', '3476', '2259', '3358', '2997', '3427'])

LabelEncoder()

In [17]:
data_train['advertiser'] = le_ad.transform(data_train.advertiser)

In [18]:
le_user= LabelEncoder()

In [19]:
data_train['useragent']= le_user.fit_transform(data_train.useragent)

In [20]:
le_adexchange= LabelEncoder()

In [21]:
le_adexchange.fit(['3', '2', '1', 'null', '4'])

LabelEncoder()

In [22]:
data_train['slotprice'] = le.fit_transform(data_train.slotprice)

In [23]:
data_train['adexchange']= le_user.fit_transform(data_train.adexchange)

In [24]:
data_train['slotwidth']=le_user.fit_transform(data_train.slotwidth)

In [25]:
data_train['slotheight']=le_user.fit_transform(data_train.slotheight)

In [26]:
data_train['slotvisibility']=le_user.fit_transform(data_train.slotvisibility)

In [27]:
data_train['slotid']=le_user.fit_transform(data_train.slotformat)

In [28]:
data_train['slotformat']=le_user.fit_transform(data_train.slotformat)

In [29]:
data_train['city']=le_user.fit_transform(data_train.city)

In [30]:
data_train['region']=le_user.fit_transform(data_train.region)

In [31]:
data_train['creative']=le_user.fit_transform(data_train.creative)

#### delete if necessary

In [32]:
data_train['usertag']= le_user.fit_transform(data_train.usertag)

In [33]:
data_train['domain']= le_user.fit_transform(data_train.domain)

In [34]:
data_train['IP']= le_user.fit_transform(data_train.IP)

In [35]:
#third drop- to improve performance 
#data_train = data_train.drop(['slotwidth','slotheight','slotvisibility','slotformat'],1)

In [36]:
data_train.city.unique()

array([  9,   4, 224, 103, 113,  13,  75, 262, 216, 106,   1, 314, 107,
       211, 228, 124, 250, 196, 255, 105, 170,  47,   2,  97,   0, 213,
       215, 349,  78, 283, 252,  84, 202, 270, 147, 148, 198, 248, 134,
       221,  64, 310,  89, 182,  96, 350, 217, 225,  14,  55, 145, 233,
       242, 179, 153, 164, 125, 166,  12, 151, 319, 128,   7, 230, 220,
       143,  74,  30, 214,  90, 227, 281, 243, 315, 308, 136, 174,  32,
        85, 146,  98,  94, 183, 234, 155, 152,  81,  80, 363,  29, 222,
       236, 154, 360, 115, 239, 208, 311, 126,  41,  82, 264, 132,  79,
        99, 149, 210, 186, 197,   5,  83, 190,  17, 192, 200, 100,  11,
       207, 127,  88, 165, 284, 235, 282, 130, 273, 119, 185,  42, 118,
       229, 150, 176,  18, 356,  95,  52,  20, 104,  26, 307,  23, 274,
       257, 168, 256, 110, 312, 159, 265, 121, 171,   6,  92, 244, 251,
       258,  49, 204, 316, 109, 156, 218, 269,  58,  76, 291, 334, 187,
       268,  34, 101,  19, 219, 322, 237,  62,  35, 226,  45,  4

In [37]:
data_train

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0,5,10,5,157545,3,9,2,3876,0,20,3,0,0,77,95,0,426282
1,0,1,16,5,30312,3,4,2,2325,3,20,3,7,3,67,12,3,512727
2,0,0,23,5,442918,19,224,2,5957,3,7,10,7,3,7,9,3,235107
3,0,1,18,5,54720,11,103,2,1145,3,20,3,7,3,47,12,3,520147
4,0,1,3,5,376910,12,113,2,2157,0,14,3,0,0,17,103,6,520147
5,0,4,17,5,427780,3,13,2,5957,3,7,10,7,3,47,25,2,164583
6,0,4,23,5,81690,8,75,1,16098,0,2,13,0,0,145,52,6,252495
7,0,4,11,5,210957,23,262,1,14173,0,7,10,1,0,2,119,8,201802
8,0,0,11,5,442561,19,216,1,5245,3,14,3,8,3,2,82,1,123474
9,0,5,13,5,224311,12,106,2,3876,0,20,3,0,0,77,96,6,457330


In [38]:
data_train.columns

Index(['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag'],
      dtype='object')

In [39]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
data_train = pd.DataFrame(mm.fit_transform(data_train))
data_train.columns = ['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag']

In [40]:
#One hot encoding - arrange in order - only run once then restart because it crashes - not necessary for random forests
#data_train = pd.get_dummies(data_train, columns = ['weekday','advertiser','hour','adexchange','slotformat','slotvisibility','useragent'])# columns = ['weekday','advertiser','hour','slotheight','slotwidth']

In [41]:
#Features and gold labels
#x_feat= data_train.ix[:,1:] # might have different columns when one hot another sample
#y_targets = data_train['click']

#### Validation pre-processing

In [42]:
#Drop columns
df_v = df_v.drop(['bidid','userid','logtype','url','urlid','keypage','bidprice','payprice'],1)

In [43]:
#Removing strings in useragent
df_v.update(df_v['useragent'].str.replace('_ie', ''))
df_v.update(df_v['useragent'].str.replace('_firefox', ''))
df_v.update(df_v['useragent'].str.replace('_safari', ''))
df_v.update(df_v['useragent'].str.replace('_chrome', ''))
df_v.update(df_v['useragent'].str.replace('_other', ''))
df_v.update(df_v['useragent'].str.replace('_opera', ''))
df_v.update(df_v['useragent'].str.replace('_maxthon', ''))
df_v.update(df_v['useragent'].str.replace('_theworld', ''))
df_v.update(df_v['useragent'].str.replace('_sogou', ''))

In [44]:
#Encoder transform for the validation set
df_v['advertiser'] = le.fit_transform(df_v.advertiser)
df_v['useragent']= le_user.fit_transform(df_v.useragent)
df_v['slotprice'] = le.fit_transform(df_v.slotprice)
df_v['adexchange']= le_user.fit_transform(df_v.adexchange)
df_v['slotwidth']=le_user.fit_transform(df_v.slotwidth)
df_v['slotheight']=le_user.fit_transform(df_v.slotheight)
df_v['slotvisibility']=le_user.fit_transform(df_v.slotvisibility)
df_v['slotid']=le_user.fit_transform(df_v.slotformat)
df_v['slotformat']=le_user.fit_transform(df_v.slotformat)
df_v['city']=le_user.fit_transform(df_v.city)
df_v['region']=le_user.fit_transform(df_v.region)
df_v['creative']=le_user.fit_transform(df_v.creative)
#Additional
df_v['usertag']= le_user.fit_transform(df_v.usertag)
df_v['domain']= le_user.fit_transform(df_v.domain)
df_v['IP']= le_user.fit_transform(df_v.IP)


In [45]:
df_v.shape

(299749, 18)

In [46]:
df_v.head()

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0,5,14,5,103240,19,226,0,6592,1,7,10,0,1,0,51,7,5214
1,0,3,14,0,49519,19,211,4,6315,3,8,0,5,3,115,13,4,114127
2,0,1,13,5,186114,15,148,2,810,0,20,3,0,0,67,112,7,114127
3,0,5,18,5,166083,15,147,2,3159,0,20,3,0,0,67,106,7,114127
4,0,5,14,5,129221,22,250,1,6315,0,9,11,2,0,2,128,6,114127


In [47]:
df_v.columns

Index(['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag'],
      dtype='object')

In [48]:
df_v = pd.DataFrame(mm.fit_transform(df_v))
df_v.columns = ['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag']

In [49]:
#Separate the training set 

X_train = data_train.ix[:,1:]
y_train = data_train['click']
#Need to repeat encoding for the validation set + validation test set
X_test = df_v.ix[:,1:]
y_test = df_v['click']

In [50]:
y_train.shape

(1618643,)

In [51]:
#Work with x_train-Bid request and y_train - clicks
X_train.head()

,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0.833333,0.434783,1.0,0.347078,0.088235,0.024390,0.5,0.200652,0.0,1.00,0.230769,0.0,0.0,0.274021,0.742188,0.000,0.819541
1,0.166667,0.695652,1.0,0.066779,0.088235,0.010840,0.5,0.120360,1.0,1.00,0.230769,0.7,1.0,0.238434,0.093750,0.375,0.985735
2,0.000000,1.000000,1.0,0.975767,0.558824,0.607046,0.5,0.308381,1.0,0.35,0.769231,0.7,1.0,0.024911,0.070312,0.375,0.452001
3,0.166667,0.782609,1.0,0.120550,0.323529,0.279133,0.5,0.059274,1.0,1.00,0.230769,0.7,1.0,0.167260,0.093750,0.375,1.000000
4,0.166667,0.130435,1.0,0.830348,0.352941,0.306233,0.5,0.111663,0.0,0.70,0.230769,0.0,0.0,0.060498,0.804688,0.750,1.000000


#### PCTR prediction feature engineering 

In [52]:
#Using random forest classifier to predict clicks
from sklearn.ensemble import RandomForestClassifier

#### Sample split is important

#Best random forest so far  #predicted 122 clicks with 7 accurate clicks [around 30mins]
clf = RandomForestClassifier(n_estimators=100, max_features = 5, oob_score = True, random_state = 2, 
                             min_impurity_split= 0.02, min_samples_leaf = 10,                                                         
                             min_weight_fraction_leaf= 0,min_samples_split = 10,class_weight= 'balanced_subsample')

In [62]:
#Feel free to change the parameters - Good so far
clf = RandomForestClassifier(n_estimators=100, max_features = 12, oob_score = True, random_state = 5, 
                             min_impurity_split= 0.04, min_samples_leaf = 12, 
                             min_weight_fraction_leaf= 0, min_samples_split = 12, class_weight= 'balanced_subsample') #balanced_subsample

In [63]:
%%time
clf.fit(X_train, y_train)

Wall time: 18min 34s


RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features=5,
            max_leaf_nodes=None, min_impurity_split=0.05,
            min_samples_leaf=15, min_samples_split=15,
            min_weight_fraction_leaf=0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=5, verbose=0, warm_start=False)

In [64]:
clf_predict = clf.predict(X_test)

In [65]:
clf.oob_score_

0.99670650044512599

In [66]:
for feature, importance in zip(X_train.columns,clf.feature_importances_):
    print (feature, ":", importance)

weekday : 0.0523150138817
hour : 0.0825283463731
useragent : 0.0483460396849
IP : 0.129871942359
region : 0.057997738833
city : 0.100559810347
adexchange : 0.0189769972342
domain : 0.104355875197
slotid : 0.0114443153402
slotwidth : 0.0302180758671
slotheight : 0.0249116883135
slotvisibility : 0.0313074504811
slotformat : 0.0105783862232
slotprice : 0.0435706818386
creative : 0.0584911500412
advertiser : 0.0324378924003
usertag : 0.162088595585


In [67]:
acc_df = pd.DataFrame([clf_predict.transpose(),y_test], index=['Predict','Test']).transpose()

In [68]:
#Number of clicks 1 = actually predicted 
np.count_nonzero(clf_predict)

307

In [69]:
print('Number of accurate clicks: '+ str(len(acc_df[(acc_df.Predict==1)&(acc_df.Test==1)])))
print('Number of clicks: '+ str(len(acc_df[acc_df.Predict==1]))+ ' out of ' + str(len(acc_df[acc_df.Test==1])))
acc_df[(acc_df.Predict==1)&(acc_df.Test==1)]

Number of accurate clicks: 20
Number of clicks: 307 out of 226


,Predict,Test
2417,1.0,1.0
41277,1.0,1.0
45304,1.0,1.0
52906,1.0,1.0
55489,1.0,1.0
92422,1.0,1.0
99680,1.0,1.0
99751,1.0,1.0
108375,1.0,1.0
116723,1.0,1.0


In [76]:
df_v.loc[[4932]]

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
4932,1.0,0.333333,0.782609,0.0,0.673227,0.323529,0.260163,1.0,0.692358,1.0,0.4,0.0,0.5,1.0,0.011321,0.101562,0.5,1.0


In [70]:
acc_df[acc_df.Test == 1]

,Predict,Test
478,0.0,1.0
1604,0.0,1.0
2417,1.0,1.0
2746,0.0,1.0
4087,0.0,1.0
4932,0.0,1.0
5910,0.0,1.0
6875,0.0,1.0
8257,0.0,1.0
8572,0.0,1.0


In [71]:
np.count_nonzero(y_train)

2034

#### Cross validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(clf, data_train.ix[:,1:], data_train['click'], cv=5)